In [8]:
from glob import glob
import pandas as pd
from tqdm import tqdm
import os
import json
import pyarrow.parquet as pq
import random

In [ ]:
# openbookqa
# https://huggingface.co/datasets/allenai/openbookqa
def format_choices(question, choice_list, answer, answer_idx):
    # example = (
    #     "The following is a multiple-choice question. Please choose the most suitable one among A, B, C and D as the answer to this question.\n\n"
    #     + question
    #     + "\n"
    # )
    example = f'Please think step by step about each option of the following multiple-choice question, and then provide the final answer.\n\nMultiple-choice question:\n{question}\n'
    for idx, choice in enumerate(["A", "B", "C", "D"]):
        example += f'{choice}. {choice_list[idx]}\n'
    ouput = answer
    return example, ouput

openbookqa_all_data = []
openbookqa_root_dir = "/xxx/openbookqa"
file_list = ["main/train-00000-of-00001.parquet","main/test-00000-of-00001.parquet","main/validation-00000-of-00001.parquet",
    "additional/train-00000-of-00001.parquet","additional/test-00000-of-00001.parquet","additional/validation-00000-of-00001.parquet"]

all_data = []
idx = 0
data_source = "openbookqa"
for half_file_dir in file_list:
    file_dir = os.path.join(openbookqa_root_dir, half_file_dir)
    table = pq.read_table(file_dir)
    df = table.to_pandas()
    for i in range(len(df)):
        question = df.iloc[i]["question_stem"]
        answer = df.iloc[i]["answerKey"]
        choice_list = df.iloc[i]["choices"]["text"]
        answer_idx = ord(answer) - ord('A')
        example, final_answer = format_choices(question, choice_list, answer, answer_idx) 
        data = {
        "data_source": data_source,
        "prompt": [{
            "role": "user",
            "content": example
        }],
        "ability": "math",
        "reward_model": {
            "style": "rule",
            "ground_truth": final_answer
        },
        "extra_info": {
            # 后边处理
            'split': None,
            'index': idx,
            'choice_list': choice_list,
        }
        }
        idx += 1
        all_data.append(data)
print(len(all_data))
        

11914


In [ ]:
# 随机打乱
random.seed(42)
random.shuffle(all_data)
train_data_num = 8000
test_data_num = 1000
test_data_50_num = 50
train_data = all_data[:train_data_num]
test_data = all_data[train_data_num:train_data_num + test_data_num]
test_data_50 = all_data[train_data_num + test_data_num:train_data_num + test_data_num + test_data_50_num]
for d in train_data:
    d["extra_info"]["split"] = "train"
for d in test_data:
    d["extra_info"]["split"] = "test"
# 保存成parquet格式
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
test_50_df = pd.DataFrame(test_data_50)
print(train_df.iloc[0])
print(len(train_df))
train_df.to_parquet("/xxx/openbookqa/openbookqa_train.parquet")
test_df.to_parquet("/xxx/openbookqa/openbookqa_test.parquet")
test_50_df.to_parquet("/xxx/openbookqa/openbookqa_test_50.parquet")

data_source                                            openbookqa
prompt          [{'role': 'user', 'content': 'Please think ste...
ability                                                      math
reward_model               {'style': 'rule', 'ground_truth': 'A'}
extra_info      {'split': 'train', 'index': 11117, 'choice_lis...
Name: 0, dtype: object
8000


In [ ]:
a = pd.read_parquet("/xxx/openbookqa/openbookqa_train.parquet")
print(a.iloc[0]["extra_info"])

[{'content': 'Please think step by step about each option of the following multiple-choice question, and then provide the final answer.\n\nMultiple-choice question:\nPhotosynthesis means plants are unable to\nA. convert sunlight to sand\nB. provide food sources for others\nC. be producers in an ecosystem\nD. make their own food\n', 'role': 'user'}]
